In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForTextToWaveform

tokenizer = AutoTokenizer.from_pretrained("sesame/csm-1b")
model = AutoModelForTextToWaveform.from_pretrained("sesame/csm-1b")